<a href="https://colab.research.google.com/github/alendoko/bina/blob/main/LB4/DATA/Python_JUPYTER_Data_Analysis_SteyByStep_Melbourne_Houseprice2025.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Strom-Endverbrauch in der Schweiz
Dieses Notebook visualisiert den Strom-Endverbrauch in der Schweiz über die Jahre, basierend auf verschiedenen Sektoren.

## Daten laden, aufbereiten und anreichern

In [ ]:
# Bibliotheken importieren
import pandas as pd
import plotly.express as px

# CSV-Daten laden
file_path = '../data/verbrauch/Strom_Endverbrauch_Schweiz.csv'
data = pd.read_csv(file_path, sep=';')

# Daten filtern
data = data[(data['Jahr'] >= 2015) & (data['Jahr'] <= 2024)]

# Spaltennamen anpassen, um " in GWh" am Ende zu entfernen
data.columns = [col.replace(' in GWh', '') for col in data.columns]

# Daten überprüfen
data.head()

## Interaktives Diagramm erstellen

In [ ]:
# Interaktives Diagramm erstellen
fig = px.line(
    data.melt(id_vars=['Jahr'], var_name='Sektor', value_name='Verbrauch (GWh)'),
    x='Jahr',
    y='Verbrauch (GWh)',
    color='Sektor',
    title='Strom-Endverbrauch in der Schweiz nach Sektoren',
    labels={'Jahr': 'Jahr', 'Verbrauch (GWh)': 'Verbrauch (GWh)', 'Sektor': 'Sektor'}
)
fig.update_traces(mode='markers+lines')
fig.update_layout(xaxis=dict(tickmode='linear'))  # Alle Jahre anzeigen

# Diagramm als HTML speichern
fig.write_html('../docs/assets/diagramme/Strom_Endverbrauch_Schweiz.html')

fig.show()